# Case Study Model

## General Remarks

- Reverse Power Flow / Cost at substation
    - If the negative generation limit at the substation is not enforced, then substation has the option of pushing energy back towards the transmission system (reverse power flow)
    - We make the follwoing assumptions: 
        - reverse power flow is allowed at the substation 
        - for scheduled negative active power the substation is remunerated according to the market price (i.e. linear cost of energy provision at the substation) -> the substation has a linear cost function 
        - While the cost function will not have a quadratic termn, THERE WILL BE a penalty on $\alpha_0$ as it indicates how much the substation is willing to deviate from the schedule


## Notes and Todos:

- Maybe check if 33bus gives nice/better results?
 
 
Done:   
- decide for a case! (combination of voltage limits and generators). Notes:
    - should there be an equal lambda case?
    - should the DERs be fully utilized in the deterministic case?
    - [Done]: Choosen Ex2 as set up
- put results handling in a function [done]
- remove unneccsray packages (e.g. gurobi from tomls) [done, but check on the "remains"]
- Remove quadratic term from substation [done]

In [176]:
# IJulia.qtconsole();

# set width of notbook 
display("text/html", "<style>.container { width:80% !important; }</style>")

In [7]:
# Set up environment from .toml-files
import Pkg
Pkg.activate(".")
# Pkg.instantiate()

# Load necessary packages
using DataFrames, CSV, LinearAlgebra, Dates # Basic Julia utility
using JLD, HDF5
using JuMP # Modeling Framework
using Mosek, MosekTools # Solver and Solver-Environment
using Logging # For some warnings in the functions

# Load functions and model
include("src/tools.jl") # Some additional functions
include("src/input.jl") # Type definitions and read-in functions
include("src/model_definition.jl") # Model definiton
include("src/output.jl") # Postprocessing of solved model

results_to_df (generic function with 1 method)

# 15Bus Case

- 15 bus case from Papavasilou paper
    - negative load is positive load
    - no line constraints
    - voltage at root is 1 p.u.
    
- Assumptions for all cases except mentioned differently
    - Two DERs one each at node 6 and 11 with `c1 = 10`, `c2 = 5`, `p_max = 1`, `q_max = 0.5`
    - No reverse power flow limit on substation
    - Substation pays `c1 = 50` for energy provision from the transmission network (no quadratic term) and a penalty of `200` on alpha to discourage high alpha

## Experiment 1

We analyse the influence of the uncertainty in the system using the "standard" setup

### Description

Ex1:
- tag: `experiment1`
- tight bascase voltage (0.95-1.05) 
- all nodes have 20% standard deviation of their forecasted load

Runs:
- `ex1_no_uncert`: deterministic case
- `ex1_uncert`: case with uncertainty
- `ex1_gen_uncert`: Chance constraints only in generation


### Notes and Observations

- Recursive decomposition of prices for active power (eqtns (20) and (21)) are verified by the experimet
- Decompositon via voltage constraints (eqtns (23) and (24)) also verified by the experiment

In [2]:
experiment1 = Dict()

# Load feeder data 
feeder = load_feeder("data/feeder_data/basecase_lv_noneg/")

# Set up
relative_std = 0.2
loads = [b.d_P for b in feeder.buses]
std_vec = loads .* relative_std
var_vec = std_vec.^2
Σ = diagm(0 => var_vec)

# ex1_general_settings
ex1_general_settings = Dict(
    "var_vec" => var_vec,
    "Σ" => Σ,
    "z_g" => 1.945,
    "z_v" => 2.576,
    "toggle_volt_cc" => false,
    "toggle_gen_cc" => false,
    "vfac" => 0, # decrease upper bound by factor (1-vfac) and increase lower bound by (1+vfac)
    "qcfac" => 1,
    "output_level" => 0,
    "Ψ" => 0,
)


# ex1_no_uncert:
ex1_no_uncert_settings = copy(ex1_general_settings)
ex1_no_uncert_settings["toggle_volt_cc"] = false
ex1_no_uncert_settings["toggle_gen_cc"] = false

experiment1["no_uncert"] = Dict(
    "verbose" => "ex1_no_uncert",
    "feeder" => feeder,
    "settings" => ex1_no_uncert_settings,
)

# ex1_uncert:
ex1_uncert_settings = copy(ex1_general_settings)
ex1_uncert_settings["toggle_volt_cc"] = true
ex1_uncert_settings["toggle_gen_cc"] = true
experiment1["uncert"] = Dict(
    "verbose" => "ex1_uncert",
    "feeder" => feeder,
    "settings" => ex1_uncert_settings,
)

# ex1_gen_uncert:
ex1_uncert_settings = copy(ex1_general_settings)
ex1_uncert_settings["toggle_volt_cc"] = false
ex1_uncert_settings["toggle_gen_cc"] = true
experiment1["gen_uncert"] = Dict(
    "verbose" => "ex1_gen_uncert",
    "feeder" => feeder,
    "settings" => ex1_uncert_settings,
)

# Run 
experiment1 = run_experiment(experiment1)

# Show results
sleep(.2) # just to ensure correct printing order
for case in keys(experiment1)
    display(experiment1[case]["results"])
end

>>>>> Reading feeder data from data/feeder_data/basecase_lv_noneg/
Running ex1_uncert
>>>> Building Model
>>>> Running Model
MOSEK warning 705: #3 (nearly) zero elements are specified in sparse row ''(87) of matrix 'A'.
MOSEK warning 705: #4 (nearly) zero elements are specified in sparse row ''(93) of matrix 'A'.
>>>> Model finished with status OPTIMAL in 10.725489949 seconds
>>>> Post-Processing

Running ex1_gen_uncert
>>>> Building Model
>>>> Running Model
>>>> Model finished with status OPTIMAL in 0.004067965 seconds
>>>> Post-Processing

Running ex1_no_uncert
>>>> Building Model
>>>> Running Model
>>>> Model finished with status OPTIMAL in 0.003565615 seconds
>>>> Post-Processing



,objective,bus,gp,gq,voltage,alpha,lambda,pi,gamma,eta,delta_plus,delta_minus,mu_plus,mu_minus,voltvar,lambda_anc,rx_pi_i,rx_pi_a,r_sum_mu_d
,Float64,Any,Any,Any,Any,Any,Any,Any,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,107.833,1,0.894374,0.540247,1.0,0.550625,-100.0,-5.22936e-10,-14.1658,0,0,0,0,0,0,0,0,0,0
2,0.0,2,0,0,0.950269,0,-101.064,-127.648,0.0,-0.0980513,0,0,-2.11416e-5,-697.74,9.92585e-5,-100.0,-1.06373,-4.3578e-12,1.06373
3,0.0,3,0,0,0.9719,0,-71.7705,-85.7815,0.0,-5.09823,0,0,-3.21486e-5,-0.000365687,0.00595755,-101.064,-60.0199,-89.3132,-29.2933
4,0.0,4,0,0,1.00487,0,-25.8566,-20.1618,0.0,-12.9354,0,0,-0.000137341,-0.000100472,0.0127409,-71.7705,-14.1072,-60.0211,-45.9139
5,0.0,5,0,0,1.00884,0,-22.6819,-15.6242,0.0,-13.4717,0,0,-0.000179682,-8.24556e-5,0.012421,-25.8566,-10.9312,-14.1059,-3.17471
6,0.0,6,0,0,1.01286,0,-19.7731,-11.4522,0.0,-13.9631,0,0,-0.000248564,-6.91739e-5,0.0119264,-22.6819,-7.98458,-10.8933,-2.90876
7,0.0,7,0.511526,-0.146374,1.02575,0.107176,-11.7616,-1.01055e-5,0.0,-15.3165,-2.53398e-5,-1.07972e-5,-83.1073,-4.68272e-5,0.00977028,-19.7731,-7.06947e-6,-8.01153,-8.01154
8,0.0,8,0,0,0.996791,0,-19.1194,-10.5279,0.0,-14.0973,0,0,-0.000119038,-0.0002906,0.0150436,-19.1194,-7.37091,-7.37089,1.79454e-5
9,0.0,9,0,0,1.00721,0,-19.1194,-10.5278,0.0,-14.0973,0,0,-0.000214085,-0.000100536,0.013504,-25.8566,-7.36225,-14.0994,-6.73718


,objective,bus,gp,gq,voltage,alpha,lambda,pi,gamma,eta,delta_plus,delta_minus,mu_plus,mu_minus,voltvar,lambda_anc,rx_pi_i,rx_pi_a,r_sum_mu_d
,Float64,Any,Any,Any,Any,Any,Any,Any,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,97.719,1,0.787837,0.543266,1.0,0.502155,-100.0,0.0,-14.0557,0,0,0,0,0,0,0,0,0,0
2,0.0,2,0,0,0.95,0,-100.923,-110.735,0.0,0,0,0,-6.22946e-6,-605.292,0,-100.0,-0.922792,0.0,0.922792
3,0.0,3,0,0,0.980882,0,-75.5108,-74.4157,0.0,0,0,0,-8.69654e-6,-2.45659e-5,0,-100.923,-52.0674,-77.4795,-25.412
4,0.0,4,0,0,1.02742,0,-35.6803,-17.4905,0.0,0,0,0,-2.43829e-5,-9.02386e-6,0,-75.5108,-12.238,-52.0684,-39.8304
5,0.0,5,0,0,1.0318,0,-32.9263,-13.554,0.0,0,0,0,-3.01897e-5,-8.4906e-6,0,-35.6803,-9.48286,-12.2369,-2.75407
6,0.0,6,0,0,1.03618,0,-30.4029,-9.93482,0.0,0,0,0,-4.04363e-5,-8.01029e-6,0,-32.9263,-6.92666,-9.45002,-2.52336
7,0.0,7,0.508279,-0.125555,1.05,0.249567,-23.4529,-2.65458e-6,0.0,0,-11.8748,-5.44797e-6,-72.0959,-6.77542e-6,0,-30.4029,-1.85705e-6,-6.95005,-6.95005
8,0.0,8,0,0,1.02254,0,-29.8358,-9.13295,0.0,0,0,0,-0.0,-9.83932e-6,0,-29.8358,-6.39429,-6.39429,1.02919e-6
9,0.0,9,0,0,1.0327,0,-29.8358,-9.13295,0.0,0,0,0,-3.00228e-5,-0.0,0,-35.6803,-6.38679,-12.2313,-5.84452


,objective,bus,gp,gq,voltage,alpha,lambda,pi,gamma,eta,delta_plus,delta_minus,mu_plus,mu_minus,voltvar,lambda_anc,rx_pi_i,rx_pi_a,r_sum_mu_d
,Float64,Any,Any,Any,Any,Any,Any,Any,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,89.9317,1,0.787837,0.543266,1.0,0,-100.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0
2,0.0,2,0,0,0.95,0,-101.074,-128.828,0.0,0,0,0,-2.34877e-6,-704.189,0,-100.0,-1.07356,0.0,1.07356
3,0.0,3,0,0,0.980882,0,-71.5095,-86.5743,0.0,0,0,0,-3.29177e-6,-1.09804e-5,0,-101.074,-60.5746,-90.1386,-29.564
4,0.0,4,0,0,1.02742,0,-25.1714,-20.3482,0.0,0,0,0,-9.08044e-6,-3.7435e-6,0,-71.5095,-14.2376,-60.5757,-46.3382
5,0.0,5,0,0,1.0318,0,-21.9673,-15.7686,0.0,0,0,0,-1.11327e-5,-3.49286e-6,0,-25.1714,-11.0322,-14.2363,-3.20404
6,0.0,6,0,0,1.03618,0,-19.0317,-11.558,0.0,0,0,0,-1.45833e-5,-3.27079e-6,0,-21.9673,-8.05839,-10.994,-2.93564
7,0.0,7,0.507154,-0.124769,1.05,0,-10.9461,-5.80027e-7,0.0,0,-1.88969e-6,-9.27997e-7,-83.8755,-2.71542e-6,0,-19.0317,-4.05766e-7,-8.0856,-8.0856
8,0.0,8,0,0,1.02254,0,-18.3719,-10.6252,0.0,0,0,0,-0.0,-4.13976e-6,0,-18.3719,-7.43904,-7.43904,4.33019e-7
9,0.0,9,0,0,1.0327,0,-18.3719,-10.6252,0.0,0,0,0,-1.08137e-5,-0.0,0,-25.1714,-7.43031,-14.2298,-6.79944


In [3]:
save_experiment("experiment1", experiment1)

Saving ex1_uncert
Saving ex1_gen_uncert
Saving ex1_no_uncert
>>>> Saved with timestamp 190320_0927


## Experiment 2

Same as Experiment1 but with wider voltage constraints

### Description

Ex2:
- tag: `experiment2` 
- bascase voltage (0.9-1.1) and `vfac = 0.02`: Resulting "real" voltage limits: ca. 0.91 - 1.09 (i.e. +- 9%)
- all nodes have 20% standard deviation of their forecasted load
- all parameters as in AP's paper, including the negative load (i.e. uncontrollable generator with the same standard deviation
- substation balancing participation penalty is set to 1000 (i.e. quadratic cost term)

Runs:
- `ex2_no_uncert`: deterministic case
- `ex2_uncert`: case with uncertainty
- `ex2_gen_uncert`: chance constraints only in generation


### Notes and Observations

- Without uncertainty the network is "uncongested" so that price for active power is the same at all nodes
- With uncertainty, some voltage constraints become binding and there are differences in the prices

In [21]:
experiment2 = Dict()

# Load feeder data 
feeder = load_feeder("data/feeder_data/basecase_noneg/")

# Set up
relative_std = 0.2
loads = [b.d_P for b in feeder.buses]
std_vec = abs.(loads) .* relative_std
var_vec = std_vec.^2
Σ = diagm(0 => var_vec)

# ex2_general_settings
ex2_general_settings = Dict(
    "var_vec" => var_vec,
    "Σ" => Σ,
    "z_g" => 1.945,
    "z_v" => 2.576,
    "toggle_volt_cc" => false,
    "toggle_gen_cc" => false,
    "vfac" => 0.02, # decrease upper bound by factor (1-vfac) and increase lower bound by (1+vfac)
    "qcfac" => 1,
    "output_level" => 0,
    "Ψ" => 0,
)

# ex2_no_uncert:
ex2_no_uncert_settings = copy(ex2_general_settings)
ex2_no_uncert_settings["toggle_volt_cc"] = false
ex2_no_uncert_settings["toggle_gen_cc"] = false

experiment2["no_uncert"] = Dict(
    "verbose" => "ex2_no_uncert",
    "feeder" => feeder,
    "settings" => ex2_no_uncert_settings,
)

# ex2_uncert:
ex2_uncert_settings = copy(ex2_general_settings)
ex2_uncert_settings["toggle_volt_cc"] = true
ex2_uncert_settings["toggle_gen_cc"] = true
experiment2["uncert"] = Dict(
    "verbose" => "ex2_uncert",
    "feeder" => feeder,
    "settings" => ex2_uncert_settings,
)

# ex2_gen_uncert:
ex2_uncert_settings = copy(ex2_general_settings)
ex2_uncert_settings["toggle_volt_cc"] = false
ex2_uncert_settings["toggle_gen_cc"] = true
experiment2["gen_uncert"] = Dict(
    "verbose" => "ex2_gen_uncert",
    "feeder" => feeder,
    "settings" => ex2_uncert_settings,
)

# Run 
experiment2 = run_experiment(experiment2)

# Show results
sleep(.2) # just to ensure correct printing order
for case in keys(experiment2)
    print(">>> Case $case")
    sleep(.2)
    display(experiment2[case]["results"])
end

>>>>> Reading feeder data from data/feeder_data/basecase_noneg/
Running ex2_uncert
>>>> Building Model
>>>> Running Model
MOSEK warning 705: #3 (nearly) zero elements are specified in sparse row ''(87) of matrix 'A'.
MOSEK warning 705: #4 (nearly) zero elements are specified in sparse row ''(93) of matrix 'A'.
>>>> Model finished with status OPTIMAL in 0.008786553 seconds
>>>> Post-Processing

Running ex2_gen_uncert
>>>> Building Model
>>>> Running Model
>>>> Model finished with status OPTIMAL in 0.003514764 seconds
>>>> Post-Processing

Running ex2_no_uncert
>>>> Building Model
>>>> Running Model
>>>> Model finished with status OPTIMAL in 0.003145126 seconds
>>>> Post-Processing

>>> Case uncert

,objective,bus,gp,gq,voltage,alpha,lambda,pi,gamma,eta,delta_plus,delta_minus,mu_plus,mu_minus,voltvar,lambda_anc,rx_pi_i,rx_pi_a,r_sum_mu_d
,Float64,Any,Any,Any,Any,Any,Any,Any,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,49.8096,1,0.112216,0.864584,1.0,0.743252,-99.9999,-4.11489e-10,-31.7738,0,0,0,0,0,0,0,0,0,0
2,0.0,2,0,0,0.90925,0,-100.742,-89.0168,0.0,-0.15797,0,0,-2.286e-5,-493.405,0.000103836,-99.9999,-0.741807,-3.42907e-12,0.741807
3,0.0,3,0,0,0.96155,0,-79.108,-58.0975,0.0,-0.290981,0,0,-4.11598e-5,-0.00022701,0.0124506,-100.742,-40.6498,-62.2836,-21.6337
4,0.0,4,0,0,1.03824,0,-45.1996,-9.63589,0.0,-0.49946,0,0,-0.000243661,-4.58485e-5,0.0150243,-79.108,-6.7422,-40.6506,-33.9084
5,0.0,5,0,0,1.04401,0,-43.6823,-7.46719,0.0,-0.506389,0,0,-0.000299971,-4.1589e-5,0.0138159,-45.1996,-5.2243,-6.74159,-1.5173
6,0.0,6,0,0,1.04963,0,-42.2921,-5.47327,0.0,-0.512737,0,0,-0.000387774,-3.81067e-5,0.0126071,-43.6823,-3.81602,-5.20621,-1.39018
7,0.0,7,0.751658,-0.241089,1.06682,0.0413561,-38.4632,-3.43147e-5,0.0,-0.530222,-26.6568,-1.21759e-5,-39.7192,-3.05569e-5,0.00925934,-42.2921,-2.40053e-5,-3.8289,-3.82893
8,0.0,8,0,0,1.05847,0,-29.8024,-12.3671,0.0,-0.605806,0,0,-82.7795,-3.58953e-5,0.0127016,-38.4612,-8.65865,-7.88567e-5,-8.65873
9,0.0,9,0,0,1.04883,0,-38.4612,-0.000112631,0.0,-0.546003,0,0,-0.000447166,-3.97714e-5,0.013294,-45.1996,-7.87642e-5,-6.7385,-6.73842


>>> Case gen_uncert

,objective,bus,gp,gq,voltage,alpha,lambda,pi,gamma,eta,delta_plus,delta_minus,mu_plus,mu_minus,voltvar,lambda_anc,rx_pi_i,rx_pi_a,r_sum_mu_d
,Float64,Any,Any,Any,Any,Any,Any,Any,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,42.8944,1,0,0.86584,1.0,0.839252,-38.6202,0.0,-31.6213,0,0,0,0,0,0,0,0,0,0
2,0.0,2,0,0,0.909207,0,-38.6202,-1.5814e-5,0.0,0,0,0,-3.63776e-9,-8.6491e-5,0,-38.6202,-1.31783e-7,0.0,1.31783e-7
3,0.0,3,0,0,0.971598,0,-38.6201,-1.06156e-5,0.0,0,0,0,-5.35071e-9,-8.90328e-9,0,-38.6202,-7.42756e-6,-1.10648e-5,-3.63721e-6
4,0.0,4,0,0,1.06204,0,-38.6201,-2.46652e-6,0.0,0,0,0,-1.64685e-8,-4.66121e-9,0,-38.6201,-1.72582e-6,-7.42771e-6,-5.70189e-6
5,0.0,5,0,0,1.06732,0,-38.6201,-1.90953e-6,0.0,0,0,0,-1.82791e-8,-4.50577e-9,0,-38.6201,-1.33597e-6,-1.72566e-6,-3.89692e-7
6,0.0,6,0,0,1.07249,0,-38.6201,-1.39811e-6,0.0,0,0,0,-2.12491e-8,-4.35955e-9,0,-38.6201,-9.74779e-7,-1.33134e-6,-3.56566e-7
7,0.0,7,0.706049,-0.22334,1.08842,0.0803751,-38.6201,-3.41191e-9,0.0,0,-27.039,-1.74234e-9,-1.01747e-5,-3.94815e-9,0,-38.6201,-2.38685e-9,-9.78069e-7,-9.80456e-7
8,0.0,8,0,0,1.08673,0,-38.6201,-1.20077e-6,0.0,0,0,0,-3.80208e-7,-0.0,0,-38.6201,-8.40702e-7,-8.80472e-7,-3.97698e-8
9,0.0,9,0,0,1.07734,0,-38.6201,-1.25758e-6,0.0,0,0,0,-0.0,-4.17802e-9,0,-38.6201,-8.79439e-7,-1.72487e-6,-8.45432e-7


>>> Case no_uncert

,objective,bus,gp,gq,voltage,alpha,lambda,pi,gamma,eta,delta_plus,delta_minus,mu_plus,mu_minus,voltvar,lambda_anc,rx_pi_i,rx_pi_a,r_sum_mu_d
,Float64,Any,Any,Any,Any,Any,Any,Any,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,15.4628,1,0,0.866397,1.0,0,-10.9503,0.0,0.0,0,0,0,0,0,0,0,0,0,0
2,0.0,2,0,0,0.909134,0,-10.9503,-6.56174e-5,0.0,0,0,0,-0.0,-0.000358983,0,-10.9503,-5.46812e-7,0.0,5.46812e-7
3,0.0,3,0,0,0.971456,0,-10.9503,-4.40179e-5,0.0,0,0,0,-2.73129e-8,-6.85084e-8,0,-10.9503,-3.07986e-5,-4.59114e-5,-1.51128e-5
4,0.0,4,0,0,1.06181,0,-10.9502,-1.01475e-5,0.0,0,0,0,-1.32139e-7,-2.60757e-8,0,-10.9503,-7.10018e-6,-3.07992e-5,-2.3699e-5
5,0.0,5,0,0,1.06711,0,-10.9502,-7.84837e-6,0.0,0,0,0,-1.48726e-7,-2.52059e-8,0,-10.9502,-5.49099e-6,-7.09954e-6,-1.60855e-6
6,0.0,6,0,0,1.07231,0,-10.9502,-5.74071e-6,0.0,0,0,0,-1.66585e-7,-2.43584e-8,0,-10.9502,-4.00249e-6,-5.47197e-6,-1.46948e-6
7,0.0,7,0.706034,-0.222345,1.0883,0,-10.9502,-2.5249e-8,0.0,0,-1.02959e-7,-4.71879e-9,-4.18648e-5,-2.18415e-8,0,-10.9502,-1.76633e-8,-4.016e-6,-4.03366e-6
8,0.0,8,0,0,1.08642,0,-10.9502,-4.71347e-6,0.0,0,0,0,-2.55645e-6,-0.0,0,-10.9502,-3.30006e-6,-3.56746e-6,-2.67404e-7
9,0.0,9,0,0,1.07703,0,-10.9502,-5.0954e-6,0.0,0,0,0,-0.0,-2.27158e-8,0,-10.9502,-3.56328e-6,-7.09628e-6,-3.533e-6


In [22]:
save_experiment("experiment2", experiment2)

Saving ex2_uncert
Saving ex2_gen_uncert
Saving ex2_no_uncert
>>>> Saved with timestamp 190320_0953


## Experiment 3

Same as before but instead of tighter voltage limits try higher load

In [10]:
experiment3 = Dict()

# Load feeder data 
feeder = load_feeder("data/feeder_data/basecase_noneg/")

# Set up
relative_std = 0.2
loads = [b.d_P for b in feeder.buses]
std_vec = loads .* relative_std
var_vec = std_vec.^2
Σ = diagm(0 => var_vec)

# ex3_general_settings
ex3_general_settings = Dict(
    "var_vec" => var_vec,
    "Σ" => Σ,
    "z_g" => 1.945,
    "z_v" => 2.576,
    "toggle_volt_cc" => false,
    "toggle_gen_cc" => false,
    "vfac" => 0, # decrease upper bound by factor (1-vfac) and increase lower bound by (1+vfac)
    "qcfac" => 1,
    "output_level" => 0,
    "Ψ" => 0,
    "loadfac" => 3
)

# ex3_no_uncert:
ex3_no_uncert_settings = copy(ex3_general_settings)
ex3_no_uncert_settings["toggle_volt_cc"] = false
ex3_no_uncert_settings["toggle_gen_cc"] = false

experiment3["no_uncert"] = Dict(
    "verbose" => "ex3_no_uncert",
    "feeder" => feeder,
    "settings" => ex3_no_uncert_settings,
)

# ex3_uncert:
ex3_uncert_settings = copy(ex3_general_settings)
ex3_uncert_settings["toggle_volt_cc"] = true
ex3_uncert_settings["toggle_gen_cc"] = true
experiment3["uncert"] = Dict(
    "verbose" => "ex3_uncert",
    "feeder" => feeder,
    "settings" => ex3_uncert_settings,
)

# ex3_gen_uncert:
ex3_uncert_settings = copy(ex3_general_settings)
ex3_uncert_settings["toggle_volt_cc"] = false
ex3_uncert_settings["toggle_gen_cc"] = true
experiment3["gen_uncert"] = Dict(
    "verbose" => "ex3_gen_uncert",
    "feeder" => feeder,
    "settings" => ex3_uncert_settings,
)

# Run 
experiment3 = run_experiment(experiment3)

# Show results
sleep(.2) # just to ensure correct printing order
for case in keys(experiment3)
    print(">>> Case $case")
    sleep(.2)
    display(experiment3[case]["results"])
end

>>>>> Reading feeder data from data/feeder_data/basecase_noneg/
Running ex3_uncert
>>>> Building Model
>>>> Running Model
MOSEK warning 705: #3 (nearly) zero elements are specified in sparse row ''(87) of matrix 'A'.
MOSEK warning 705: #4 (nearly) zero elements are specified in sparse row ''(93) of matrix 'A'.
>>>> Model finished with status OPTIMAL in 0.006354567 seconds
>>>> Post-Processing

Running ex3_gen_uncert
>>>> Building Model
>>>> Running Model
>>>> Model finished with status OPTIMAL in 0.002476817 seconds
>>>> Post-Processing

Running ex3_no_uncert
>>>> Building Model
>>>> Running Model
>>>> Model finished with status OPTIMAL in 0.00223549 seconds
>>>> Post-Processing

>>> Case uncert

,objective,bus,gp,gq,voltage,alpha,lambda,pi,gamma,eta,delta_plus,delta_minus,mu_plus,mu_minus,voltvar,lambda_anc,rx_pi_i,rx_pi_a,r_sum_mu_d
,Float64,Any,Any,Any,Any,Any,Any,Any,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,431.923,1,3.8177,1.0676,1.0,1.0,-100.0,-7.68782e-10,-31.6228,0,0,0,0,0,0,0,0,0,0
2,0.0,2,0,0,0.916252,0,-100.0,-0.000145897,0.0,-1.5754e-8,0,0,-3.26811e-6,-0.000764702,0.00363129,-100.0,-1.21581e-6,-6.40651e-12,1.1877e-6
3,0.0,3,0,0,0.950524,0,-100.0,-0.000106729,0.0,-5.32418e-6,0,0,-4.5928e-6,-5.34742e-5,0.0117104,-100.0,-7.46768e-5,-0.000102082,-2.95957e-5
4,0.0,4,0,0,1.00189,0,-99.9999,-2.54546e-5,0.0,-1.30342e-5,0,0,-1.39688e-5,-1.24571e-5,0.0211438,-100.0,-1.78105e-5,-7.46783e-5,-5.99181e-5
5,0.0,5,0,0,1.0105,0,-99.9999,-1.88947e-5,0.0,-1.3443e-5,0,0,-1.80855e-5,-9.65875e-6,0.0212127,-99.9999,-1.32194e-5,-1.78089e-5,-4.72767e-6
6,0.0,6,0,0,1.01952,0,-99.9999,-1.32239e-5,0.0,-1.37819e-5,0,0,-2.39752e-5,-7.56658e-6,0.0205562,-99.9999,-9.21983e-6,-1.31736e-5,-4.0367e-6
7,0.0,7,0.8,-0.0477213,1.04946,0,-99.9999,-2.49905e-7,0.0,-1.45891e-5,-88.4176,-2.06216e-6,-0.000103338,-4.4126e-6,0.0150349,-99.9999,-1.74825e-7,-9.25095e-6,-9.53643e-6
8,0.0,8,0,0,0.970333,0,-99.9999,-2.22249e-5,0.0,-1.47582e-5,0,0,-7.50706e-6,-5.08665e-5,0.0177988,-99.9999,-1.55604e-5,-1.08896e-5,4.53539e-6
9,0.0,9,0,0,1.00209,0,-99.9999,-1.55536e-5,0.0,-1.43558e-5,0,0,-1.67668e-5,-1.18817e-5,0.0209381,-99.9999,-1.08768e-5,-1.78007e-5,-7.42324e-6


>>> Case gen_uncert

,objective,bus,gp,gq,voltage,alpha,lambda,pi,gamma,eta,delta_plus,delta_minus,mu_plus,mu_minus,voltvar,lambda_anc,rx_pi_i,rx_pi_a,r_sum_mu_d
,Float64,Any,Any,Any,Any,Any,Any,Any,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,431.923,1,3.8177,1.06751,1.0,1.0,-100.0,0.0,-31.6228,0,0,0,0,0,0,0,0,0,0
2,0.0,2,0,0,0.916265,0,-100.0,-1.46431e-5,0.0,0,0,0,-5.19691e-7,-7.75417e-5,0,-100.0,-1.22026e-7,0.0,1.22026e-7
3,0.0,3,0,0,0.950548,0,-100.0,-1.06024e-5,0.0,0,0,0,-6.95592e-7,-3.72078e-6,0,-100.0,-7.41829e-6,-1.02455e-5,-2.82723e-6
4,0.0,4,0,0,1.00193,0,-100.0,-3.07235e-6,0.0,0,0,0,-1.49108e-6,-9.49878e-7,0,-100.0,-2.14971e-6,-7.41843e-6,-5.26872e-6
5,0.0,5,0,0,1.01167,0,-100.0,-2.38737e-6,0.0,0,0,0,-1.87868e-6,-8.13124e-7,0,-100.0,-1.67029e-6,-2.14952e-6,-4.79231e-7
6,0.0,6,0,0,1.02172,0,-100.0,-1.81109e-6,0.0,0,0,0,-2.49437e-6,-7.01822e-7,0,-100.0,-1.26271e-6,-1.6645e-6,-4.01791e-7
7,0.0,7,0.8,-0.00573882,1.05434,0,-100.0,-4.76193e-7,0.0,0,-88.4189,-3.44537e-7,-1.01621e-5,-4.74925e-7,0,-100.0,-3.33128e-7,-1.26698e-6,-9.33847e-7
8,0.0,8,0,0,0.96786,0,-100.0,-2.67144e-6,0.0,0,0,0,-0.0,-1.95064e-6,0,-100.0,-1.87037e-6,-1.66633e-6,2.04037e-7
9,0.0,9,0,0,0.999696,0,-100.0,-2.38002e-6,0.0,0,0,0,-1.41964e-6,-0.0,0,-100.0,-1.66437e-6,-2.14853e-6,-4.84157e-7


>>> Case no_uncert

,objective,bus,gp,gq,voltage,alpha,lambda,pi,gamma,eta,delta_plus,delta_minus,mu_plus,mu_minus,voltvar,lambda_anc,rx_pi_i,rx_pi_a,r_sum_mu_d
,Float64,Any,Any,Any,Any,Any,Any,Any,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,399.29,1,3.8177,1.07126,1.0,0,-100.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0
2,0.0,2,0,0,0.915774,0,-100.0,-7.20018e-6,0.0,0,0,0,-0.0,-3.84094e-5,0,-100.0,-6.00015e-8,0.0,6.00015e-8
3,0.0,3,0,0,0.949578,0,-100.0,-5.07783e-6,0.0,0,0,0,-2.51789e-7,-1.51122e-6,0,-100.0,-3.55287e-6,-5.03784e-6,-1.48497e-6
4,0.0,4,0,0,1.00027,0,-100.0,-1.25314e-6,0.0,0,0,0,-6.13942e-7,-0.0,0,-100.0,-8.76815e-7,-3.55294e-6,-2.67613e-6
5,0.0,5,0,0,1.01009,0,-100.0,-9.36669e-7,0.0,0,0,0,-7.81308e-7,-3.25326e-7,0,-100.0,-6.55326e-7,-8.76736e-7,-2.21411e-7
6,0.0,6,0,0,1.02022,0,-100.0,-6.68596e-7,0.0,0,0,0,-1.05106e-6,-2.82462e-7,0,-100.0,-4.66152e-7,-6.53056e-7,-1.86904e-7
7,0.0,7,0.8,-0.00323173,1.05305,0,-100.0,-3.86404e-8,0.0,0,-89.0495,-8.43607e-8,-4.76745e-6,-1.9593e-7,0,-100.0,-2.70315e-8,-4.67726e-7,-4.40694e-7
8,0.0,8,0,0,0.965763,0,-100.0,-9.90414e-7,0.0,0,0,0,-0.0,-7.79882e-7,0,-100.0,-6.93422e-7,-6.11847e-7,8.15756e-8
9,0.0,9,0,0,0.997666,0,-100.0,-8.739e-7,0.0,0,0,0,-6.07748e-7,-0.0,0,-100.0,-6.11129e-7,-8.76334e-7,-2.65205e-7


## Experiment 4

- Instead of uncertain load uncontrollable vs. controllable DERs
- Maybe better in the 33-bus system?

## Prototyping Space

In [124]:
results = experiment2["uncert"]["results"]
feeder = experiment2["uncert"]["feeder"]
results[results[:bus] .== 3, :lambda][1]



# lambda_a = []
# rx_pi = []
# rx_pi_a = []

# for bus in feeder.buses
#     if bus.is_root
#         push!(lambda_a, 0)
#         push!(rx_pi, 0)
#         push!(rx_pi_a, 0)
#     else
#         anc = bus.ancestor
#         cs = bus.children
#         push!(lambda_a, results[results[:bus] .== anc, :lambda][1])
        
#         pi_i =  results[results[:bus] .== bus.index, :pi][1]
#         push!(rx_pi, pi_i * feeder.line_to[bus.index].r / feeder.line_to[bus.index].x)
        
#         pi_a = pi_i =  results[results[:bus] .== anc, :pi][1]
#         push!(rx_pi_a, pi_a * feeder.line_to[bus.index].r / feeder.line_to[bus.index].x)
#     end
# end


r_sum_mu_d = []

for bus in feeder.buses
    downstream = traverse(feeder, bus.index)
    downstream = setdiff(downstream, [feeder.root_bus])
    v = 2*sum(feeder.line_to[d].r * (results[results[:bus] .== d, :mu_plus][1] - results[results[:bus] .== d, :mu_minus][1]) for d in downstream)
    push!(r_sum_mu_d, v)
end
    
r_sum_mu_d


15-element Array{Any,1}:
 -7.205953103912941     
 -7.206062392432603     
 -8.414541366815063     
 -8.414580810945816     
 -6.937927467672224     
 -6.93791763606441      
 -6.937905206646532     
 -2.4250666387669306e-5 
 -1.476600361745382     
 -1.4765493195107668    
 -1.4765406443863434    
 -1.4764544415890568    
  0.00010928851966197678
  0.00010889073586762795
  4.2411773376212617e-5 

In [120]:
function traverse(feeder, bus_idx; nodes=[])
    push!(nodes, bus_idx)
    if length(feeder.buses[bus_idx].children) > 0
        for c in feeder.buses[bus_idx].children
            nodes = traverse(feeder, c, nodes=nodes)
        end
    end
    return nodes 
end

traverse (generic function with 2 methods)

In [121]:
traverse(feeder, 9)

5-element Array{Any,1}:
  9
  8
 10
 11
 12

In [23]:
feeder.generators

3-element Array{Generator,1}:
 Generator("g1", 1, 1000.0, 1000.0, 50.0, 1000.0)
 Generator("g2", 12, 0.8, 0.4, 10.0, 5.0)        
 Generator("g3", 7, 0.8, 0.4, 10.0, 5.0)         

In [ ]:
# datadir = ("data/feeder_data/basecase_noneg/")
datadir = ("data/feeder_data/basecase_lv_noneg/")
# datadir = ("data/feeder_data/feeder141/")
# datadir = ("data/feeder_data/feeder141_lv/")

feeder = load_feeder(datadir);

In [ ]:
mu = results_df[:mu_plus] .+ results_df[:mu_minus]
mu = mu[2:end]

b = 1 ./ diag(C)
b = [(x == Inf ? 0 : x) for x in b]

g3 = 2*s*z*e'*R'*mu/sum(b)

g1 = s^2/sum(b) 

delta = results_df[:delta_plus] .+ results_df[:delta_minus]
detlta = delta[2:end]
g2 = sum(delta.*b)/sum(b)

g = -g1 - g2 + g3


In [ ]:
saveres = results_df

In [ ]:
CSV.write("figures/results_uncert.csv", results_df)

In [ ]:
shadow_price(m[:δp][12])

In [ ]:
m.dictList

In [20]:
A = [1 2; 3 4]

2×2 Array{Int64,2}:
 1  2
 3  4

In [31]:
feeder.buses()

MethodError: MethodError: objects of type Array{Bus,1} are not callable
Use square brackets [] for indexing an Array.

In [144]:

Dates.format(Dates.now(), "yymmdd_HHMM")

"190318_1923"